#ESTRAZIONE DATI DA TWITTER

In [3]:
# DEFINISCO DEI API KEY
API_KEY = "XXXXXXXXXXXXXXXXXXX"
API_KEY_SECRET = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
BEARER_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [4]:
#IMPORTO LA LIBRERIA TWEEPY
import tweepy as tw
# SE NON CI FOSSE LA INSTALLEREI CON 
# ! pip install tweepy

In [5]:
# autenticazione
auth = tw.OAuthHandler(API_KEY, API_KEY_SECRET)

# oggetto della classe API necessario per eseguire le API
api = tw.API(auth, wait_on_rate_limit=True)
# passiamo l'handler di autenticazione
# passiamo il parametro wait_on_rate_limit: potrebbe capitare di fare piu di un certo numero di operazioni ogni 15 minuti
# L'API non mi darà piu risultati se non aspetto quei 15 minuti. Settando il parametro mi stoppa l'esecuzione se supero il limite

A questo punto abbiamo l'accesso alle API

In [35]:

#definisco la query di ricerca
search_query = "#BritishGP"

Il cursore è un oggetto su cui andare ad iterare per ottenere le risposte dall'API. Si usa questo metodo perche altrimenti il numero di risultati forniti dall'API sarebbe eccessivamente grande e non gestibile. 

In [36]:
#definisco il cursore
cursor = tw.Cursor(
    api.search,
    q=search_query, 
    lang="it", 
    since="2022-07-01", 
    country="Italy"
)

questo cursore ci fornirà risultati relativi al metodo search dell'oggetto API
- parametro q è la query di ricerca 
- lingua dei risultati (meglio usare l'inglese)
- dati pubblicati dalla data yyy-mm-dd
- paese di riferimento per la pubblicazione dei tweet (Italia)

Per ricavare i dati dal cursore devo specificare al cursore quanti elementi restituire. (100 per non sovraccaricare le ricerche)

In [37]:
#ricavo i tweet
tweets = cursor.items(300)

Un oggetto tweet è un item iterator

In [38]:
tweets

Per avere effettivamente una lista di tweets mi vado a mettere ogni tweet all'interno di una lista

In [39]:
tweets_copy = []
for tweet in tweets: 
    tweets_copy.append(tweet)

verifico la lunghezza della lista appena creata:

In [40]:
len(tweets_copy)

4

Vogliamo ora creare il nostro dataset fatto di features e attributi associati a ciascuna features. Lo creiamo usando pandas. 

Dobbiamo creare un dataframe ovvero una struttura dati in forma tabellare. Definiamo i nomi delle features che useremo per le nostre analisi. Queste informazioni sono prese da tweetter. 

In [ ]:
# creiamo il dataset con features e attributi associati usando pandas
import pandas as pd 

#definisco il dataframe pandas
tweets_df = pd.DataFrame(columns=['user_name', 'user_location', 'user_description', 
                                  'user_verified', 'id', 'date', 'text', 
                                  'in_reply_to_status_id', 'in_reply_to_user_id', 
                                  'retweet_original_user', 'retweet_original_tweet_id', 
                                  'hashtags', 'retweet_count', 'favorite_count', 'source'])


Per effettuare il retreival dei dati e metterli correttamente all'interno della struttura creata. 

Dobbiamo immaginare ogni tweet come un oggetto che ha varie proprietà; qui andiamo ad estrarre solo le proprietà di interesse dal tweet. 

- lista di hashtag relativi al tweet 

In [42]:
from tqdm import tqdm 
#prendiamo i dati dalla lista che ho definito sopra iterandovi sopra.
for tweet in tqdm(tweets_copy):

    # RICAVIAMO IL TESTO DEI TWEET
    hashtags = []
    try:
        #vado a riempire la lista di hashtag relativi a ciascun tweet
        for hashtag in tweet.entities["hashtags"]:
            #prendo solo il campo text
            hashtags.append(hashtag["text"])
    except:
        #se il try fallisce (non trovo il campo hastags) vado all'istruzione dopo
        pass

    #prendo il testo relativo ad un tweet che ha un id che ho a disposizione
    text = api.get_status(id=tweet.id, tweet_mode="extended").full_text #restituisce un oggetto di cui prendo solo il campo full_text
  
    # RICAVIAMO ALTRE INFO SUL TWEET
    try:
        #ricavare l'id dell'utente del tweet originario
        original_tweeter = tweet.retweeted_status.user.id
        
        #ricavo anche l'id del tweet originario
        original_tweet_id = tweet.retweeted_status.id
        
        #copio il tweet all'interno della lista di tweet che sto processando
        tweets_copy.append(tweet.retweeted_status) 
    except:
        # gestisco il caso in cui non siano disponibili queste info
        original_tweeter = None
        original_tweet_id = None

    # ANDIAMO AD AGGIUNGERE AL DATASET LA LISTA CHE CI INTERESSA:
    tweets_df = tweets_df.append(pd.DataFrame({'user_id': tweet.user.id,  
                                               'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'id': tweet.id,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'in_reply_to_status_id': [tweet.in_reply_to_status_id if tweet.in_reply_to_status_id else None],
                                               'in_reply_to_user_id': [tweet.in_reply_to_user_id if tweet.in_reply_to_user_id else None],
                                               'retweet_original_user': original_tweeter, 
                                               'retweet_original_tweet_id': original_tweet_id,
                                               'hashtags': [hashtags if hashtags else None],
                                               'retweet_count': tweet.retweet_count,
                                               'favorite_count': tweet.favorite_count,
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True) # quando creo il df viene fornito un indice; cosi lo elimino




5it [00:00,  8.97it/s]


In [ ]:
#vediamo le prime righe del df 
tweets_df.head()

In [44]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user_name                  5 non-null      object        
 1   user_location              5 non-null      object        
 2   user_description           5 non-null      object        
 3   user_verified              5 non-null      object        
 4   id                         5 non-null      object        
 5   date                       5 non-null      datetime64[ns]
 6   text                       5 non-null      object        
 7   in_reply_to_status_id      0 non-null      object        
 8   in_reply_to_user_id        0 non-null      object        
 9   retweet_original_user      1 non-null      object        
 10  retweet_original_tweet_id  1 non-null      object        
 11  hashtags                   3 non-null      object        
 12  retweet_coun

Ho piu di 1000 elementi perche vi erano degli elementi gia nella lista. 
Per togliere i duplicati dal df: 

In [45]:
tweets_df = tweets_df.drop_duplicates("id") #specifico la colonna sulla quale effettua la ricerca dei duplicati

In [46]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user_name                  5 non-null      object        
 1   user_location              5 non-null      object        
 2   user_description           5 non-null      object        
 3   user_verified              5 non-null      object        
 4   id                         5 non-null      object        
 5   date                       5 non-null      datetime64[ns]
 6   text                       5 non-null      object        
 7   in_reply_to_status_id      0 non-null      object        
 8   in_reply_to_user_id        0 non-null      object        
 9   retweet_original_user      1 non-null      object        
 10  retweet_original_tweet_id  1 non-null      object        
 11  hashtags                   3 non-null      object        
 12  retweet_coun

Ho i 100 tweet della lista piu 12 tweet che sono tweet originari di retweet che stanno nei 100 considerati

In [47]:
# SHAPE DEL DF
tweets_df.shape #(righe, colonne)

(5, 16)

In [48]:
# andiamo infine a considerare le occorrenze di ciascun giorno 
pd.to_datetime(tweets_df["date"]).dt.floor("d").value_counts().rename_axis("date").reset_index(name="count")

,date,count
0,2022-08-21,2
1,2022-08-20,1
2,2022-08-19,1
3,2018-08-21,1


In [ ]:
 #SALVATAGGIO DEL CSV DA SALVARE: 
 tweets_df.to_csv("/content/drive/MyDrive/Colab Notebooks/BritishGB_tweet_1000.csv", index=False) #escludo la colonna con l'indice

se non specifico il path mi saLva il file nella root ovvero dove si trovano le cartelle drive e sample_data. Lo possiamo caricare su drive se specifichiamo un path sul drive.

#Parte 2: Analisi dei dati

Alcune domande a cui possiamo rispondere analizzando questi dati: 
- quali sono le possibili locazioni degli utenti che hanno pubblicato
- il periodo di riferimento dei tweet
- quanti sono i retweet
- quanti sono stati retweetati
- l'hastag piu usato